In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.llms import OpenAI


In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


### LLM CHAIN

In [24]:
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.chat_models import ChatOpenAI


In [72]:
# rep = ["pomegranate seeds","coriander"]
rep = ["chicken stock","coriander","apricot"]

In [25]:
llm = ChatOpenAI(temperature=0.6,model_name = "gpt-3.5-turbo")

In [55]:
# # This is an LLMChain to write a synopsis given a title of a play and the era it is set in.

# template = """You are an expert chef, specializing in ingredient substitutions. You have the title, 
# ingredients, and instructions of a specific recipe. The user will provide you with certain ingredients 
# they wish to replace in this recipe. Your task is to suggest ideal replacements for each specified 
# ingredient, ensuring that the substitutes belong to the same food group and have a similar flavor profile. 
# Along with each substitution, please provide a brief explanation as to why the chosen replacement is suitable, 
# considering factors such as taste, texture, and culinary use. Focus solely on providing ingredient replacements 
# without altering the original recipe or offering any additional information or guidance.

# Title: {title}
# Ingredients: {ingredients}
# Instructions: {instructions}
# Ingredients that require replacement: {replacements}
# Suggestions: Here are the suggestions:"""
# prompt_template = PromptTemplate(input_variables=["title", "ingredients","instructions","replacements"], template=template)
# llm_chain = LLMChain(llm = llm, prompt = prompt_template)


In [56]:
# suggestions = llm_chain(inputs={"title":title,"ingredients":ing,"instructions":ins,"replacements":rep})

In [57]:
suggestions = get_suggestions(title,ing,ins,rep)

In [58]:
formatted_suggestions = [line for line in suggestions['text'].split('- ') if line.strip()]
for topic in formatted_suggestions:
    print(f'- {topic}')

- Chicken stock replacement: Vegetable stock
  
- Explanation: Vegetable stock is a suitable replacement for chicken stock as it belongs to the same food group and provides a similar flavor profile. Both stocks add depth and richness to the dish. Vegetable stock can also enhance the savory notes in the tagine without overpowering the other ingredients.


- Coriander replacement: Parsley
  
- Explanation: Parsley is a suitable replacement for coriander as it belongs to the same herb family and has a similar fresh and slightly citrusy flavor profile. It can provide a similar aromatic and vibrant element to the dish, complementing the other ingredients in the tagine.


- Apricot replacement: Dried figs
  
- Explanation: Dried figs are a suitable replacement for apricots as they belong to the same fruit group and offer a similar sweet and slightly tangy flavor profile. Dried figs will provide a similar burst of sweetness and a pleasant chewy texture, adding depth to the tagine.


In [18]:
formatted_suggestions = [line for line in suggestions['text'].split('- ') if line.strip()]
for topic in formatted_suggestions:
    print(f'- {topic}')

- Ras-el-hanout spice mix: A suitable replacement for ras-el-hanout spice mix would be Moroccan spice blend. This spice blend shares similar flavors and profiles with ras-el-hanout and is commonly used in Moroccan cuisine. It contains a combination of spices like cumin, coriander, ginger, paprika, cinnamon, and cloves, which will provide a similar aromatic and warm flavor to the dish.


- Dried apricot: A suitable replacement for dried apricot would be dried figs. Dried figs have a similar sweet and slightly tangy flavor profile to dried apricots and will provide a similar fruity element to the dish. They also have a soft and chewy texture, which will add a pleasant bite to the tagine.


In [73]:
formatted_suggestions

['Chicken stock replacement: Vegetable stock\n  ',
 'Explanation: Vegetable stock is a suitable replacement for chicken stock as it belongs to the same food group and provides a similar flavor profile. Both stocks add depth and richness to the dish. Vegetable stock can also enhance the savory notes in the tagine without overpowering the other ingredients.\n\n',
 'Coriander replacement: Parsley\n  ',
 'Explanation: Parsley is a suitable replacement for coriander as it belongs to the same herb family and has a similar fresh and slightly citrusy flavor profile. It can provide a similar aromatic and vibrant element to the dish, complementing the other ingredients in the tagine.\n\n',
 'Apricot replacement: Dried figs\n  ',
 'Explanation: Dried figs are a suitable replacement for apricots as they belong to the same fruit group and offer a similar sweet and slightly tangy flavor profile. Dried figs will provide a similar burst of sweetness and a pleasant chewy texture, adding depth to the ta

In [16]:
def get_suggestions(title,ing,ins,rep):

    llm = ChatOpenAI(temperature=0.6,model_name = "gpt-3.5-turbo")

    template = """You are an expert chef, specializing in ingredient substitutions. You have the title, 
    ingredients, and instructions of a specific recipe. The user will provide you with certain ingredients 
    they wish to replace in this recipe. Your task is to suggest ideal replacements for each specified 
    ingredient, ensuring that the substitutes belong to the same food group and have a similar flavor profile. 
    Along with each substitution, please provide a brief explanation as to why the chosen replacement is suitable, 
    considering factors such as taste, texture, and culinary use. Focus solely on providing ingredient replacements 
    without altering the original recipe or offering any additional information or guidance.

    Title: {title}
    Ingredients: {ingredients}
    Instructions: {instructions}
    Ingredients that require replacement: {replacements}
    Suggestions: Here are the suggestions:"""
    
    prompt_template = PromptTemplate(input_variables=["title", "ingredients","instructions","replacements"], template=template)
    
    llm_chain = LLMChain(llm = llm, prompt = prompt_template)

    output = llm_chain(inputs={"title":title,"ingredients":ing,"instructions":ins,"replacements":rep})

    return output

In [60]:
def replace_ingredients(output, ingredients, replacement_list):
    # Iterate through the replacement list
    for initial_ingredient in replacement_list:
        # Check if the initial ingredient is mentioned in any of the sentences in the output
        for sentence in output:
            if initial_ingredient.lower() in sentence.lower():
                # Extract the replacement ingredient from the sentence
                replacement = sentence.split(":")[1].split(".")[0].strip().lower()
                if 'could be' in replacement:
                    replacement = replacement.split('could be')[1].strip()
                elif 'would be' in replacement:
                    replacement = replacement.split('would be')[1].strip()
                # Replace the initial ingredient with the replacement ingredient in the original ingredients list
                ingredients = [ingredient.lower().replace(initial_ingredient.lower(), replacement) for ingredient in ingredients]
                break  # Move to the next initial ingredient
    return ingredients

In [74]:
new_ingredients = replace_ingredients(formatted_suggestions, ing, rep)

In [6]:
new_ingredients 

['2 tbsp olive oil',
 '500g lean diced lamb',
 '1 large onion, roughly chopped',
 '2 large carrots, quartered lengthways and cut into chunks',
 '2 garlic cloves, finely chopped',
 '2 tbsp ras-el-hanout spice mix',
 '400g can chopped tomato',
 '400g can chickpea, rinsed and drained',
 '200g dried dried figs',
 '600ml vegetable stock',
 '120g pack pomegranate seeds',
 '2 large handfuls parsley, roughly chopped']

In [7]:
message = '\n'.join(new_ingredients)

In [10]:
from datetime import datetime
import pywhatkit
import time,webbrowser,pyautogui

In [ ]:
# pywhatkit.sendwhatmsg_to_group_instantly(group_link, message)

In [12]:

group_link = ""

# BLOG

In [5]:
import pandas as pd

In [60]:
# Read the CSV file with tab separator
# df = pd.read_csv('en.openfoodfacts.org.products.csv', sep='\t', encoding='utf-8')

In [6]:
# df = df.dropna(subset=['product_name'])

In [7]:
# df['product_name'].nunique()

In [8]:
# all_products = df['product_name'].unique()

In [9]:
# all_products_df = pd.DataFrame(all_products, columns=['product_name'])
# all_products_df.to_csv('all_products.csv', index=False)

In [65]:
# ingredients_input = "cookie',milk"

In [66]:
# type(ingredients_input)


str

In [12]:
ingredients_input = "1 tsp butter, 2 cookies"

In [21]:
[item.split()[-1] for item in ingredients_input.split(',')]

['butter', 'cookies']

In [15]:
[item.strip() for item in ingredients_input.split(',')]] 

['2 cookies']

In [72]:
def check_ingredients(ingredients_input):
    all_products_loaded_df = pd.read_csv('all_products.csv')
    all_products_loaded = all_products_loaded_df['product_name'].values
    ingredients_list = [item.strip().lower() for item in ingredients_input] 
    return all(any(ingredient in product for product in all_products_loaded) for ingredient in ingredients_list)


In [73]:
check_ingredients(ingredients_input)

True

In [19]:
from langchain.prompts import ChatPromptTemplate

In [37]:
## Router Chain

comfort_template = """
You are an expert food blogger, who values family traditions and comfort cooking.\
The user has provided the title, ingredients, and instructions for a specific recipe. The user will also provide some \
Create a heartfelt 300-word blog post focusing on family memories, passed-down secrets, and the joy of bonding over cooking. \
The post should concentrate on the narrative and not include the recipe details.


Title: {title}
Ingredients: {ingredients}
Instructions: {instructions}

Write Blog Post:
"""


seasonal_farm_two_table_template = """
You are an expert food blogger, passionate about seasonal and farm-to-table cuisine. \
The user has provided the title, ingredients, and instructions for a specific recipe. User also talks about \
Compose a 300-word blog post highlighting the seasonality of the ingredients and the farm-to-table philosophy.\
Discuss freshness, support for local farmers, and seasonal availability. \
The post should consist solely of the narrative, without the recipe details.

Title: {title}
Ingredients: {ingredients}
Instructions: {instructions}

Write Blog Post:
"""

healthy_living_template = """
You are an expert food blogger, with a focus on healthy living and nutrition.\
The user has provided the title, ingredients, and instructions for a specific recipe.\
Craft a 300-word blog post emphasizing its nutritional benefits and alignment with a healthy lifestyle.\
Explore aspects such as nutritional value, diet-friendly variations, and organic ingredient options. \
The post should contain only the narrative, excluding the recipe details.

    Title: {title}
    Ingredients: {ingredients}
    Instructions: {instructions}

Write Blog Post:
"""


cultural_exploration_template = """ 
You are an expert food blogger, specializing in culinary cultural exploration. \
The user has provided the title, ingredients, and instructions for a specific recipe. \
Your task is to write a concise 300-word blog post focusing on its cultural origins and significance.\
Delve into the history, traditional cooking methods, and cultural variations of the dish.\
Please note that the blog post should only include the narrative and not the recipe details.

Title: {title}
Ingredients: {ingredients}
Instructions: {instructions}

Write Blog Post:
"""

In [38]:
# theme = ["Cultural Exploration", "Healthy Living and Nutritional Focus", "Seasonal and Farm-to-Table Experience","Family and Comfort Cooking"]

In [39]:
theme_templates = {
    "Cultural Exploration": cultural_exploration_template,
    "Healthy Living and Nutritional Focus": healthy_living_template,
    "Seasonal and Farm-to-Table Experience": seasonal_farm_two_table_template,
    "Family and Comfort Cooking": comfort_template
}


In [50]:
selected_theme = "Healthy Living and Nutritional Focus"

In [41]:
title = "Sautéed Spinach"

In [42]:
ing = "1 pounds fresh spinach,2 tablespoons olive oil,1 garlic"
ins = "Discard blemished spinach leaves and tough stems. Rinse the spinach and drain well.Heat the oil in a skillet large enough to hold all the spinach. Add the spinach, garlic, salt and pepper. Cook over high heat, stirring rapidly, until the spinach is wilted and most of the moisture has evaporated. Remove the garlic and serve immediately."
theme = "Healthy Living and Nutritional Focus"

In [51]:
def generate_blog_post(title,ing,ins,selected_theme):
    selected_template = theme_templates[theme]
    prompt_template = PromptTemplate(input_variables=["title", "ingredients","instructions"], template=selected_template)
    llm_chain = LLMChain(llm = llm, prompt = prompt_template)
    output = llm_chain(inputs={"title":title,"ingredients":ing,"instructions":ins})
    return output

In [52]:
blog_post = generate_blog_post(title,ing,ins,theme)

In [53]:
blog_post['text']

"When it comes to incorporating nutritious and delicious greens into our diet, spinach is a clear winner. Today, I want to share with you a simple yet incredibly flavorful recipe for sautéed spinach that will not only tantalize your taste buds but also provide you with a wide range of health benefits.\n\nSpinach is a powerhouse of nutrients, packed with vitamins A, C, and K, as well as folate, iron, and calcium. These essential vitamins and minerals are crucial for maintaining healthy bones, boosting our immune system, and promoting overall well-being. By including spinach in our diet, we can improve our cardiovascular health, lower blood pressure, and reduce the risk of developing chronic diseases such as diabetes and certain types of cancer.\n\nThe beauty of this sautéed spinach recipe lies in its simplicity. With just a few ingredients and minimal cooking time, you can create a flavorful side dish that complements any main course. To make it even more diet-friendly, you can use a li

In [46]:
blog_post['text']

"When it comes to incorporating nutrient-dense foods into your diet, spinach should be at the top of your list. This leafy green is not only delicious, but it also offers a wide array of health benefits that can support your journey towards a healthy lifestyle.\n\nOne of the easiest and tastiest ways to enjoy spinach is through sautéing. This simple cooking method helps to retain the nutritional value of the spinach while adding a burst of flavor. With just a few ingredients and a quick cooking time, you can have a vibrant and nutritious side dish ready in no time.\n\nSpinach is packed with essential vitamins and minerals that are crucial for optimal health. It is an excellent source of vitamin K, which plays a vital role in blood clotting and bone health. Additionally, spinach contains high levels of vitamin A, which is essential for maintaining healthy vision and a robust immune system.\n\nFor those following a specific diet or looking to make healthier choices, sautéed spinach can e

In [48]:
bl = [line for line in blog_post['text'].split('- ') if line.strip()]

In [49]:
for line in bl:
    print(line)

When it comes to incorporating nutrient-dense foods into your diet, spinach should be at the top of your list. This leafy green is not only delicious, but it also offers a wide array of health benefits that can support your journey towards a healthy lifestyle.

One of the easiest and tastiest ways to enjoy spinach is through sautéing. This simple cooking method helps to retain the nutritional value of the spinach while adding a burst of flavor. With just a few ingredients and a quick cooking time, you can have a vibrant and nutritious side dish ready in no time.

Spinach is packed with essential vitamins and minerals that are crucial for optimal health. It is an excellent source of vitamin K, which plays a vital role in blood clotting and bone health. Additionally, spinach contains high levels of vitamin A, which is essential for maintaining healthy vision and a robust immune system.

For those following a specific diet or looking to make healthier choices, sautéed spinach can easily b